In [1]:
import ast
import requests
import pandas as pd
from mapbox import Geocoder

In [2]:
import re
from tqdm import tqdm
tqdm.pandas()

### Data Preperation

In [3]:
df = pd.read_csv("./data/combined_NER_files.csv")

In [4]:
df

,Unnamed: 0.1,Unnamed: 0,position_section,position_subsection,hl1,hl2,author,lede,body,pub_date,indexing_terms,year,actual_body_word_count,body_cleaned,llama_pred,span_NER_pred
0,0,0,LIVING ARTS,unknown,greater boston author readings nov. 13-19,unknown,unknown,"sunday katrina kenison (""moments of seeing: re...","monday john simpson (""the word detective: sear...",2016-11-13,"{'subject': [{'score': '90', 'classCode': 'ST0...",2016,359,monday john simpson ( `` word detective : sear...,Based on the information provided in the hea...,"[(Boston, 'GPE'), (Massachusetts, 'GPE'), (1, ..."
1,1,1,SPORTS,unknown,tough time breaking the ice,bc has only all-female staff in division 1 hockey,unknown,the boston college women's hockey team takes t...,it's unacceptable that women's college hockey ...,2016-11-13,"{'subject': [{'score': '90', 'classCode': 'STX...",2016,1002,' unacceptable women ' college hockey ' boast ...,Based on the information provided in the art...,"[(Boston, 'GPE'), (Allston, 'LOC'), (1, 'CARDI..."
2,2,2,MAGAZINE,unknown,top 5 places new englanders like to go in winter,our favorite destinations aren't all warm — pl...,By Marisa Dellatto,> $141.17 — average cost of a night in a mass....,> 25 degrees — average temperature across new ...,2016-11-13,"{'subject': [{'score': '91', 'classCode': 'ST0...",2016,176,25 degrees average temperature across new engl...,Based on the information provided in the art...,"[(1, 'CARDINAL'), (Boston, 'GPE'), (Massachuse..."
3,3,3,SUNDAY,Travel,home design with art in mind,unknown,unknown,the restored sunken garden at hill-stead in fa...,"hill-stead, pope's 33,000-square-foot colonial...",2016-11-13,"{'subject': [{'score': '90', 'classCode': 'N71...",2016,794,"hill- stead , pope ' 33 , 000- square- foot co...",Based on the information provided in the art...,"[(Boston, 'GPE'), (Massachusetts, 'GPE'), (1, ..."
4,4,4,NEWS,Metro,at least the hens won,unknown,unknown,"well, i'm happy for the chickens. since tuesda...",earlier this year i went to visit the country ...,2016-11-13,"{'subject': [{'score': '90', 'classCode': 'ST0...",2016,562,earlier year went visit country hen hubbardsto...,Based on the information provided in the art...,"[(1, 'CARDINAL'), (Boston, 'GPE'), (2, 'CARDIN..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,145,145,SPORTS,unknown,vonn sliding on a slippery slope,concussed skier stubbornly races on,John Powers,what will it take to keep fuzzy-thinking linds...,on a treacherous day when olympic champion and...,2011-02-09,"{'subject': [{'score': '99', 'classCode': 'STX...",2011,1292,treacherous day olympic champion andrea fischb...,Based on the information provided in the art...,"[(1.Y, 'CARDINAL'), (Boston, 'GPE'), (Boston, ..."
1496,146,146,REGIONAL,South,"for centenarian, a natural legacy",norwell woman feels spirit of late husband in ...,unknown,"year after year, for decades before most of no...",norris has long been referred to as ``extraord...,2011-12-15,"{'subject': [{'score': '76', 'classCode': 'STX...",2011,879,"norris long referred extraordinary , inspiring...",1. Y - The article is talking about a specif...,"[(1, 'CARDINAL'), (Boston, 'GPE'), (2, 'CARDIN..."
1497,147,147,NEWS,Metro,"o'malley, bishops blast gambling bill","`moral fabric of our society' faces threat, th...",unknown,the state's four catholic bishops condemned ye...,the bill passed the house wednesday night. a s...,2011-09-17,{'legal': [{'className': 'Criminal Law & Proce...,2011,218,bill passed house wednesday night . similar ve...,Based on the information provided in the art...,"[(1, 'CARDINAL'), (Boston, 'GPE'), (Massachuse..."
1498,148,148,NEWS,Metro,officer's shooting is detailed,"2d suspect in woburn shootout held on $500,000...",unknown,woburn - as antonio matos hovered over wounded...,"baez-cruz, charged with armed robbery and assa...",2011-09-17,{'legal': [{'className': 'Criminal Law & Proce...,2011,691,"baez- cruz , charged armed robbery assault mur...",Based on the info

In [5]:
def remove_first_comma(x):
    if (x[:1] == ","):
        return x[2:]
    else:
        return x

def format_NER(x):
    res = []
    if (x != None):
        input = x.replace("(","").replace("[","").replace("]","").replace("'","").split(")") 
        for word in input:
            res.append(remove_first_comma(word).strip())
    return res 

In [6]:
df['NER_Filtered'] = df['span_NER_pred'].progress_apply(format_NER)


100%|█████████████████████████████████████████████████████████████████████| 1500/1500 [00:00<00:00, 106806.82it/s]


In [7]:
df

,Unnamed: 0.1,Unnamed: 0,position_section,position_subsection,hl1,hl2,author,lede,body,pub_date,indexing_terms,year,actual_body_word_count,body_cleaned,llama_pred,span_NER_pred,NER_Filtered
0,0,0,LIVING ARTS,unknown,greater boston author readings nov. 13-19,unknown,unknown,"sunday katrina kenison (""moments of seeing: re...","monday john simpson (""the word detective: sear...",2016-11-13,"{'subject': [{'score': '90', 'classCode': 'ST0...",2016,359,monday john simpson ( `` word detective : sear...,Based on the information provided in the hea...,"[(Boston, 'GPE'), (Massachusetts, 'GPE'), (1, ...","[Boston, GPE, Massachusetts, GPE, 1, CARDINAL,..."
1,1,1,SPORTS,unknown,tough time breaking the ice,bc has only all-female staff in division 1 hockey,unknown,the boston college women's hockey team takes t...,it's unacceptable that women's college hockey ...,2016-11-13,"{'subject': [{'score': '90', 'classCode': 'STX...",2016,1002,' unacceptable women ' college hockey ' boast ...,Based on the information provided in the art...,"[(Boston, 'GPE'), (Allston, 'LOC'), (1, 'CARDI...","[Boston, GPE, Allston, LOC, 1, CARDINAL, Allst..."
2,2,2,MAGAZINE,unknown,top 5 places new englanders like to go in winter,our favorite destinations aren't all warm — pl...,By Marisa Dellatto,> $141.17 — average cost of a night in a mass....,> 25 degrees — average temperature across new ...,2016-11-13,"{'subject': [{'score': '91', 'classCode': 'ST0...",2016,176,25 degrees average temperature across new engl...,Based on the information provided in the art...,"[(1, 'CARDINAL'), (Boston, 'GPE'), (Massachuse...","[1, CARDINAL, Boston, GPE, Massachusetts, GPE,..."
3,3,3,SUNDAY,Travel,home design with art in mind,unknown,unknown,the restored sunken garden at hill-stead in fa...,"hill-stead, pope's 33,000-square-foot colonial...",2016-11-13,"{'subject': [{'score': '90', 'classCode': 'N71...",2016,794,"hill- stead , pope ' 33 , 000- square- foot co...",Based on the information provided in the art...,"[(Boston, 'GPE'), (Massachusetts, 'GPE'), (1, ...","[Boston, GPE, Massachusetts, GPE, 1, CARDINAL,..."
4,4,4,NEWS,Metro,at least the hens won,unknown,unknown,"well, i'm happy for the chickens. since tuesda...",earlier this year i went to visit the country ...,2016-11-13,"{'subject': [{'score': '90', 'classCode': 'ST0...",2016,562,earlier year went visit country hen hubbardsto...,Based on the information provided in the art...,"[(1, 'CARDINAL'), (Boston, 'GPE'), (2, 'CARDIN...","[1, CARDINAL, Boston, GPE, 2, CARDINAL, Boston..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,145,145,SPORTS,unknown,vonn sliding on a slippery slope,concussed skier stubbornly races on,John Powers,what will it take to keep fuzzy-thinking linds...,on a treacherous day when olympic champion and...,2011-02-09,"{'subject': [{'score': '99', 'classCode': 'STX...",2011,1292,treacherous day olympic champion andrea fischb...,Based on the information provided in the art...,"[(1.Y, 'CARDINAL'), (Boston, 'GPE'), (Boston, ...","[1.Y, CARDINAL, Boston, GPE, Boston, GPE, Fenw..."
1496,146,146,REGIONAL,South,"for centenarian, a natural legacy",norwell woman feels spirit of late husband in ...,unknown,"year after year, for decades before most of no...",norris has long been referred to as ``extraord...,2011-12-15,"{'subject': [{'score': '76', 'classCode': 'STX...",2011,879,"norris long referred extraordinary , inspiring...",1. Y - The article is talking about a specif...,"[(1, 'CARDINAL'), (Boston, 'GPE'), (2, 'CARDIN...","[1, CARDINAL, Boston, GPE, 2, CARDINAL, Beacon..."
1497,147,147,NEWS,Metro,"o'malley, bishops blast gambling bill","`moral fabric of our society' faces threat, th...",unknown,the state's four catholic bishops condemned ye...,the bill passed the house wednesday night. a s...,2011-09-17,{'legal': [{'className': 'Criminal Law & Proce...,2011,218,bill passed house wednesday night . similar ve...,Based on the information provided in the art...,"[(1, 'CARDINAL'), (Boston,

In [8]:
def filter_loc(x):
    res = []
    for tup in x:
        cleaned_tup = tup.strip().split(",")
        if (len(cleaned_tup) >= 2): 
            if (("GPE" in cleaned_tup[1] and "Boston" not in cleaned_tup[0] and "Massachusetts" not in cleaned_tup[0])
                or ("ORG" in cleaned_tup[1]) 
                or ("FAC" in cleaned_tup[1])
                or ("LOC" in cleaned_tup[1])
            ):
                res.append((cleaned_tup[0], cleaned_tup[1].strip()))
    priority = {'FAC': 1, 'ORG': 2, 'LOC': 3, 'GPE': 4}
    sorted_list = sorted(res, key=lambda x: priority[x[1]])
    
    return sorted_list

In [9]:
df['NER_Filtered'] = df['NER_Filtered'].progress_apply(filter_loc)

100%|█████████████████████████████████████████████████████████████████████| 1500/1500 [00:00<00:00, 329654.49it/s]


In [10]:
df

,Unnamed: 0.1,Unnamed: 0,position_section,position_subsection,hl1,hl2,author,lede,body,pub_date,indexing_terms,year,actual_body_word_count,body_cleaned,llama_pred,span_NER_pred,NER_Filtered
0,0,0,LIVING ARTS,unknown,greater boston author readings nov. 13-19,unknown,unknown,"sunday katrina kenison (""moments of seeing: re...","monday john simpson (""the word detective: sear...",2016-11-13,"{'subject': [{'score': '90', 'classCode': 'ST0...",2016,359,monday john simpson ( `` word detective : sear...,Based on the information provided in the hea...,"[(Boston, 'GPE'), (Massachusetts, 'GPE'), (1, ...","[(Fenway, LOC)]"
1,1,1,SPORTS,unknown,tough time breaking the ice,bc has only all-female staff in division 1 hockey,unknown,the boston college women's hockey team takes t...,it's unacceptable that women's college hockey ...,2016-11-13,"{'subject': [{'score': '90', 'classCode': 'STX...",2016,1002,' unacceptable women ' college hockey ' boast ...,Based on the information provided in the art...,"[(Boston, 'GPE'), (Allston, 'LOC'), (1, 'CARDI...","[(Harvard Avenue, FAC), (Brighton Avenue, FAC)..."
2,2,2,MAGAZINE,unknown,top 5 places new englanders like to go in winter,our favorite destinations aren't all warm — pl...,By Marisa Dellatto,> $141.17 — average cost of a night in a mass....,> 25 degrees — average temperature across new ...,2016-11-13,"{'subject': [{'score': '91', 'classCode': 'ST0...",2016,176,25 degrees average temperature across new engl...,Based on the information provided in the art...,"[(1, 'CARDINAL'), (Boston, 'GPE'), (Massachuse...","[(The North End, LOC)]"
3,3,3,SUNDAY,Travel,home design with art in mind,unknown,unknown,the restored sunken garden at hill-stead in fa...,"hill-stead, pope's 33,000-square-foot colonial...",2016-11-13,"{'subject': [{'score': '90', 'classCode': 'N71...",2016,794,"hill- stead , pope ' 33 , 000- square- foot co...",Based on the information provided in the art...,"[(Boston, 'GPE'), (Massachusetts, 'GPE'), (1, ...","[(South End, LOC)]"
4,4,4,NEWS,Metro,at least the hens won,unknown,unknown,"well, i'm happy for the chickens. since tuesda...",earlier this year i went to visit the country ...,2016-11-13,"{'subject': [{'score': '90', 'classCode': 'ST0...",2016,562,earlier year went visit country hen hubbardsto...,Based on the information provided in the art...,"[(1, 'CARDINAL'), (Boston, 'GPE'), (2, 'CARDIN...","[(Fenway Park, FAC), (Boston Red Sox, ORG), (F..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,145,145,SPORTS,unknown,vonn sliding on a slippery slope,concussed skier stubbornly races on,John Powers,what will it take to keep fuzzy-thinking linds...,on a treacherous day when olympic champion and...,2011-02-09,"{'subject': [{'score': '99', 'classCode': 'STX...",2011,1292,treacherous day olympic champion andrea fischb...,Based on the information provided in the art...,"[(1.Y, 'CARDINAL'), (Boston, 'GPE'), (Boston, ...","[(Tremont Street, FAC), (Boylston Street, FAC)..."
1496,146,146,REGIONAL,South,"for centenarian, a natural legacy",norwell woman feels spirit of late husband in ...,unknown,"year after year, for decades before most of no...",norris has long been referred to as ``extraord...,2011-12-15,"{'subject': [{'score': '76', 'classCode': 'STX...",2011,879,"norris long referred extraordinary , inspiring...",1. Y - The article is talking about a specif...,"[(1, 'CARDINAL'), (Boston, 'GPE'), (2, 'CARDIN...","[(Beacon Hill, LOC)]"
1497,147,147,NEWS,Metro,"o'malley, bishops blast gambling bill","`moral fabric of our society' faces threat, th...",unknown,the state's four catholic bishops condemned ye...,the bill passed the house wednesday night. a s...,2011-09-17,{'legal': [{'className': 'Criminal Law & Proce...,2011,218,bill passed house wednesday night . similar ve...,Based on the information provided in the art...,"[(1, 'CARDINAL'), (Boston, 'GPE'), (Massachuse...","[(South End, LOC)]"
1498,148,148,NEWS,Metro,officer's shooting is detailed,"2d suspect in wobur

In [11]:
df.to_csv("./test.csv")

In [12]:
empty_criteria_deletion = df['NER_Filtered'].apply(lambda x: x == [])
df = df[~empty_criteria_deletion] # Invert it so that True becomes false so it marks for deletion

In [13]:
df['year'].value_counts()

year
2016    145
2019    145
2017    145
2013    144
2014    144
2018    144
2012    144
2011    142
2015    141
2010    140
Name: count, dtype: int64

### Census Pipeline

In [29]:
# Replace 'YOUR_MAPBOX_ACCESS_TOKEN' with your actual Mapbox access token
mapbox_access_token = 'pk.eyJ1IjoiYWJkdWxyYWZheTAzIiwiYSI6ImNsdHB5YzRmczAwZTAyaXFiMnd6MXBjbXQifQ.fy4hqXUdVEsmPuC5XLyUSA'

geocoder = Geocoder(access_token=mapbox_access_token)

# Modified function to return the TRACT information
def query_census_api(longitude, latitude):
    county = "NO COUNTY"
    url = f"https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x={longitude}&y={latitude}&benchmark=Public_AR_Current&vintage=Census2020_Current&format=json"
    response = requests.get(url)
    if (response.status_code == 200):
        results = response.json()
        print(results['result']['geographies'])
        if ('Incorporated Places' in results['result']['geographies']):
            if (results['result']['geographies']['Incorporated Places'][0].get('BASENAME') == "Boston"):
                census_tracts = results['result']['geographies'].get('Census Tracts', [])
                county = results['result']['geographies']['County Subdivisions'][0].get('COUNTY')
                
                if (census_tracts):
                    return census_tracts[0].get('TRACT', 'No TRACT found'), county # Returning the TRACT of the first census tract found
    return "No TRACT found", county  # Return this if API call failed or no tracts found

In [30]:
def getTractList(x):
    # Initialize an empty list to store TRACT information
    tract_list = []
    
    location = x[0][0] # (Location, Label)
    response = geocoder.forward(f"Boston, {location}")
    if response.status_code == 200:
        # Extract the coordinates of the first matching feature
        coordinates = response.json()['features'][0]['geometry']['coordinates']
        # Query the U.S. Census Geocoding Services API with these coordinates
        TRACT, COUNTY = query_census_api(coordinates[0], coordinates[1])
        tract_list.append(TRACT)
    else:
        print(f"Failed to get coordinates for {location}")
    return tract_list

def getCountyList(x):
    # Initialize an empty list to store TRACT information
    counties_list = []
    
    location = x[0][0] # (Location, Label)
    response = geocoder.forward(f"Boston, {location}")
    if response.status_code == 200:
        # Extract the coordinates of the first matching feature
        coordinates = response.json()['features'][0]['geometry']['coordinates']
        # Query the U.S. Census Geocoding Services API with these coordinates
        TRACT, COUNTY = query_census_api(coordinates[0], coordinates[1])
        counties_list.append(COUNTY)
    else:
        print(f"Failed to get coordinates for {location}")
    return counties_list

def getLongLats(x):
    # Initialize an empty list to store TRACT information
    location = x[0][0] # (Location, Label)
    response = geocoder.forward(f"Boston, {location}")
    if response.status_code == 200:
        # Extract the coordinates of the first matching feature
        coordinates = response.json()['features'][0]['geometry']['coordinates']
        return [coordinates[0], coordinates[1]]
    else:
        print(f"Failed to get coordinates for {location}")
    return tract_list

In [31]:
# Adding the TRACT information as a new column to the DataFrame
df['TRACT_LIST'] = df['NER_Filtered'].progress_apply(getTractList)
df['COUNTY_LIST'] = df['NER_Filtered'].progress_apply(getCountyList)

  0%|                                                                            | 2/1434 [00:00<05:59,  3.98it/s]

{'2018 State Legislative Districts - Upper': [{'POP100': 185474, 'GEOID': '25028', 'CENTLAT': '+42.3706547', 'AREAWATER': 1668060, 'STATE': '25', 'BASENAME': 'Second Suffolk & Middlesex', 'OID': '212704693077277', 'LSADC': '07', 'SLDU': '028', 'FUNCSTAT': 'N', 'INTPTLAT': '+42.3713037', 'NAME': 'Second Suffolk & Middlesex District', 'OBJECTID': 1421, 'CENTLON': '-071.1603667', 'LSY': '2018', 'AREALAND': 33955670, 'INTPTLON': '-071.1600825', 'HU100': 81685, 'MTFCC': 'G5210', 'LDTYP': 'O'}], 'States': [{'STATENS': '00606926', 'POP100': 7029917, 'GEOID': '25', 'CENTLAT': '+42.1618490', 'AREAWATER': 7130705554, 'STATE': '25', 'BASENAME': 'Massachusetts', 'STUSAB': 'MA', 'OID': '27470203627104', 'LSADC': '00', 'FUNCSTAT': 'A', 'INTPTLAT': '+42.1565196', 'DIVISION': '1', 'NAME': 'Massachusetts', 'REGION': '1', 'OBJECTID': 52, 'CENTLON': '-071.4975238', 'AREALAND': 20204345052, 'INTPTLON': '-071.4895915', 'HU100': 2998537, 'MTFCC': 'G4000', 'UR': 'M'}], 'Combined Statistical Areas': [{'POP100

  0%|▏                                                                           | 3/1434 [00:00<07:30,  3.17it/s]

{'2018 State Legislative Districts - Upper': [{'POP100': 176126, 'GEOID': '25001', 'CENTLAT': '+42.3476240', 'AREAWATER': 119113331, 'STATE': '25', 'BASENAME': 'First Suffolk', 'OID': '212704693077281', 'LSADC': '07', 'SLDU': '001', 'FUNCSTAT': 'N', 'INTPTLAT': '+42.3291293', 'NAME': 'First Suffolk District', 'OBJECTID': 1334, 'CENTLON': '-070.9101087', 'LSY': '2018', 'AREALAND': 33404043, 'INTPTLON': '-070.9277651', 'HU100': 76294, 'MTFCC': 'G5210', 'LDTYP': 'O'}], 'States': [{'STATENS': '00606926', 'POP100': 7029917, 'GEOID': '25', 'CENTLAT': '+42.1618490', 'AREAWATER': 7130705554, 'STATE': '25', 'BASENAME': 'Massachusetts', 'STUSAB': 'MA', 'OID': '27470203627104', 'LSADC': '00', 'FUNCSTAT': 'A', 'INTPTLAT': '+42.1565196', 'DIVISION': '1', 'NAME': 'Massachusetts', 'REGION': '1', 'OBJECTID': 52, 'CENTLON': '-071.4975238', 'AREALAND': 20204345052, 'INTPTLON': '-071.4895915', 'HU100': 2998537, 'MTFCC': 'G4000', 'UR': 'M'}], 'Combined Statistical Areas': [{'POP100': 8466186, 'GEOID': '14

  0%|▏                                                                           | 4/1434 [00:01<08:19,  2.87it/s]

{'2018 State Legislative Districts - Upper': [{'POP100': 99338, 'GEOID': '09023', 'CENTLAT': '+41.1823804', 'AREAWATER': 7570507, 'STATE': '09', 'BASENAME': '23', 'OID': '212704692443752', 'LSADC': 'LU', 'SLDU': '023', 'FUNCSTAT': 'N', 'INTPTLAT': '+41.1827575', 'NAME': 'State Senate District 23', 'OBJECTID': 975, 'CENTLON': '-073.1797479', 'LSY': '2018', 'AREALAND': 29964076, 'INTPTLON': '-073.1798743', 'HU100': 38867, 'MTFCC': 'G5210', 'LDTYP': 'O'}], 'States': [{'STATENS': '01779780', 'POP100': 3605944, 'GEOID': '09', 'CENTLAT': '+41.5751437', 'AREAWATER': 1816359839, 'STATE': '09', 'BASENAME': 'Connecticut', 'STUSAB': 'CT', 'OID': '27470331955805', 'LSADC': '00', 'FUNCSTAT': 'A', 'INTPTLAT': '+41.5798637', 'DIVISION': '1', 'NAME': 'Connecticut', 'REGION': '1', 'OBJECTID': 33, 'CENTLON': '-072.7393081', 'AREALAND': 12541754848, 'INTPTLON': '-072.7466572', 'HU100': 1530197, 'MTFCC': 'G4000', 'UR': 'M'}], 'Combined Statistical Areas': [{'POP100': 23582649, 'GEOID': '408', 'CENTLAT': '

  0%|▎                                                                           | 5/1434 [00:01<09:04,  2.63it/s]

{'2018 State Legislative Districts - Upper': [{'POP100': 169760, 'GEOID': '25002', 'CENTLAT': '+42.3055574', 'AREAWATER': 423295, 'STATE': '25', 'BASENAME': 'Second Suffolk', 'OID': '212704693077291', 'LSADC': '07', 'SLDU': '002', 'FUNCSTAT': 'N', 'INTPTLAT': '+42.3044915', 'NAME': 'Second Suffolk District', 'OBJECTID': 724, 'CENTLON': '-071.1022674', 'LSY': '2018', 'AREALAND': 31065333, 'INTPTLON': '-071.1003111', 'HU100': 73131, 'MTFCC': 'G5210', 'LDTYP': 'O'}], 'States': [{'STATENS': '00606926', 'POP100': 7029917, 'GEOID': '25', 'CENTLAT': '+42.1618490', 'AREAWATER': 7130705554, 'STATE': '25', 'BASENAME': 'Massachusetts', 'STUSAB': 'MA', 'OID': '27470203627104', 'LSADC': '00', 'FUNCSTAT': 'A', 'INTPTLAT': '+42.1565196', 'DIVISION': '1', 'NAME': 'Massachusetts', 'REGION': '1', 'OBJECTID': 52, 'CENTLON': '-071.4975238', 'AREALAND': 20204345052, 'INTPTLON': '-071.4895915', 'HU100': 2998537, 'MTFCC': 'G4000', 'UR': 'M'}], 'Combined Statistical Areas': [{'POP100': 8466186, 'GEOID': '148'

KeyboardInterrupt: 

In [17]:
df

,Unnamed: 0.1,Unnamed: 0,position_section,position_subsection,hl1,hl2,author,lede,body,pub_date,indexing_terms,year,actual_body_word_count,body_cleaned,llama_pred,span_NER_pred,NER_Filtered,TRACT_LIST,COUNTY_LIST
0,0,0,LIVING ARTS,unknown,greater boston author readings nov. 13-19,unknown,unknown,"sunday katrina kenison (""moments of seeing: re...","monday john simpson (""the word detective: sear...",2016-11-13,"{'subject': [{'score': '90', 'classCode': 'ST0...",2016,359,monday john simpson ( `` word detective : sear...,Based on the information provided in the hea...,"[(Boston, 'GPE'), (Massachusetts, 'GPE'), (1, ...","[(Fenway, LOC)]",[010403],[025]
1,1,1,SPORTS,unknown,tough time breaking the ice,bc has only all-female staff in division 1 hockey,unknown,the boston college women's hockey team takes t...,it's unacceptable that women's college hockey ...,2016-11-13,"{'subject': [{'score': '90', 'classCode': 'STX...",2016,1002,' unacceptable women ' college hockey ' boast ...,Based on the information provided in the art...,"[(Boston, 'GPE'), (Allston, 'LOC'), (1, 'CARDI...","[(Harvard Avenue, FAC), (Brighton Avenue, FAC)...",[091900],[025]
2,2,2,MAGAZINE,unknown,top 5 places new englanders like to go in winter,our favorite destinations aren't all warm — pl...,By Marisa Dellatto,> $141.17 — average cost of a night in a mass....,> 25 degrees — average temperature across new ...,2016-11-13,"{'subject': [{'score': '91', 'classCode': 'ST0...",2016,176,25 degrees average temperature across new engl...,Based on the information provided in the art...,"[(1, 'CARDINAL'), (Boston, 'GPE'), (Massachuse...","[(The North End, LOC)]",[No TRACT found],[NO COUNTY]
3,3,3,SUNDAY,Travel,home design with art in mind,unknown,unknown,the restored sunken garden at hill-stead in fa...,"hill-stead, pope's 33,000-square-foot colonial...",2016-11-13,"{'subject': [{'score': '90', 'classCode': 'N71...",2016,794,"hill- stead , pope ' 33 , 000- square- foot co...",Based on the information provided in the art...,"[(Boston, 'GPE'), (Massachusetts, 'GPE'), (1, ...","[(South End, LOC)]",[070700],[025]
4,4,4,NEWS,Metro,at least the hens won,unknown,unknown,"well, i'm happy for the chickens. since tuesda...",earlier this year i went to visit the country ...,2016-11-13,"{'subject': [{'score': '90', 'classCode': 'ST0...",2016,562,earlier year went visit country hen hubbardsto...,Based on the information provided in the art...,"[(1, 'CARDINAL'), (Boston, 'GPE'), (2, 'CARDIN...","[(Fenway Park, FAC), (Boston Red Sox, ORG), (F...",[010206],[025]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,145,145,SPORTS,unknown,vonn sliding on a slippery slope,concussed skier stubbornly races on,John Powers,what will it take to keep fuzzy-thinking linds...,on a treacherous day when olympic champion and...,2011-02-09,"{'subject': [{'score': '99', 'classCode': 'STX...",2011,1292,treacherous day olympic champion andrea fischb...,Based on the information provided in the art...,"[(1.Y, 'CARDINAL'), (Boston, 'GPE'), (Boston, ...","[(Tremont Street, FAC), (Boylston Street, FAC)...",[070802],[025]
1496,146,146,REGIONAL,South,"for centenarian, a natural legacy",norwell woman feels spirit of late husband in ...,unknown,"year after year, for decades before most of no...",norris has long been referred to as ``extraord...,2011-12-15,"{'subject': [{'score': '76', 'classCode': 'STX...",2011,879,"norris long referred extraordinary , inspiring...",1. Y - The article is talking about a specif...,"[(1, 'CARDINAL'), (Boston, 'GPE'), (2, 'CARDIN...","[(Beacon Hill, LOC)]",[030302],[025]
1497,147,147,NEWS,Metro,"o'malley, bishops blast gambling bill","`moral fabric of our society' faces threat, th...",unknown,the state's four catholic bishops condemned ye...,the bill passed the house wednesday night. a s...,2011-09-17,{'legal': [{'className': 'Criminal Law & Proce...,2011,218,bill passed house wednesday night . similar ve...,Based on the information provided in the art..

In [18]:
def filter_tracts(x):
    res = []
    for tract in x:
        if (tract != "No TRACT found"):
            res.append(tract)
    return res

In [19]:
df['TRACT_LIST'] = df['TRACT_LIST'].progress_apply(filter_tracts)

100%|████████████████████████████████████████████████████████████████████| 1434/1434 [00:00<00:00, 1150025.23it/s]
/var/folders/cm/zwkyr_z9583424krb99hcqsc0000gn/T/ipykernel_35252/2392340556.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TRACT_LIST'] = df['TRACT_LIST'].progress_apply(filter_tracts)


In [20]:
empty_criteria_deletion = df['NER_Filtered'].apply(lambda x: len(x) == 0)
df = df[~empty_criteria_deletion] # Invert it so that True becomes false so it marks for deletion

In [21]:
df['year'].value_counts()

year
2016    145
2019    145
2017    145
2013    144
2014    144
2018    144
2012    144
2011    142
2015    141
2010    140
Name: count, dtype: int64

In [22]:
# Adding the TRACT information as a new column to the DataFrame
df['LONG_LAT'] = df['NER_Filtered'].progress_apply(getLongLats)

100%|█████████████████████████████████████████████████████████████████████████| 1434/1434 [00:17<00:00, 80.25it/s]


In [23]:
df.to_csv("./location_output_WITH_LONG_LATS_AND_COUNTY.csv")